<a id='top'></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Data Availability for Predicting Nightly Mood Scores
This notebook explores the available data that we can use to predict the mood scores as reported on the evening EMA. We focus on the overlap between the Fitbit and Beiwe data.

In [3]:
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
import matplotlib.dates as mdates

# Table of Contents
1. [Data Import](#data_import)
    1. [Targets](#targets)
    2. [Features](#features)
2. [Pre-Processing](#preprocessing)
3. [Summary](#summary)

---

<a id='data_import'></a>

# Data Import
Data are simply imported from the Fitbit daily dataset and Fitbit sleep summary dataset - both of each include data summarized on a daily basis.

<a id='targets'></a>

## Target Data

### Evening EMAs

In [4]:
ema_e = pd.read_csv("../data/processed/beiwe-evening_ema-ux_s20.csv",parse_dates=["timestamp"])
ema_e["date"] = ema_e["timestamp"].dt.date
ema_e.head()

,timestamp,beiwe,content,stress,lonely,sad,energy,redcap,beacon,date
0,2020-05-13 19:00:23,awa8uces,1.0,1.0,1.0,3.0,2.0,28,26.0,2020-05-13
1,2020-05-13 19:21:32,lkkjddam,0.0,2.0,3.0,1.0,0.0,12,21.0,2020-05-13
2,2020-05-13 19:30:38,rvhdl2la,2.0,1.0,0.0,0.0,1.0,29,NaN,2020-05-13
3,2020-05-13 20:07:04,xxvnhauv,1.0,3.0,1.0,2.0,0.0,21,22.0,2020-05-13
4,2020-05-13 20:25:07,xdbdrk6e,2.0,1.0,2.0,1.0,3.0,23,24.0,2020-05-13


In [5]:
print("Number of Datapoints:", len(ema_e))

Number of Datapoints: 2460


<a id='features'></a>

## Feature Data
The feature data is compromised of:
* morning EMA mood (Beiwe)
* morning EMA sleep (Beiwe)
* current daily acitivity (Fitbit)
* previous night's sleep (Fitbit)

In [6]:
ema_m = pd.read_csv("../data/processed/beiwe-morning_ema-ux_s20.csv",parse_dates=["timestamp"])
ema_m["date"] = ema_m["timestamp"].dt.date
ema_m.head()

,timestamp,beiwe,content,stress,lonely,sad,energy,tst,sol,naw,restful,redcap,beacon,date
0,2020-05-13 09:10:27,qh34m4r9,3,0.0,0.0,0.0,1.0,8.0,20.0,2.0,3.0,68,19.0,2020-05-13
1,2020-05-13 09:15:49,awa8uces,0.0,2.0,1.0,1.0,1.0,2.0,10.0,3.0,1.0,28,26.0,2020-05-13
2,2020-05-13 09:42:19,xxvnhauv,1,1.0,1.0,3.0,0.0,6.0,30.0,3.0,1.0,21,22.0,2020-05-13
3,2020-05-13 09:43:27,rvhdl2la,1,1.0,2.0,3.0,0.0,5.3,5.0,2.0,2.0,29,NaN,2020-05-13
4,2020-05-13 10:11:51,q5y11ytm,3,1.0,0.0,1.0,2.0,2.0,10.0,0.0,0.0,48,NaN,2020-05-13


In [7]:
print("Number of Datapoints:", len(ema_m))

Number of Datapoints: 2470


### Fitbit Activity
We need two sets of activity data:
1. **Daily Activity**: so we can see if the participant was wearing their Fitbit the entire day
2. **Hourly Activity**: to get the activity level prior to submission time

In [8]:
act_daily = pd.read_csv("../data/processed/fitbit-daily-ux_s20.csv",parse_dates=["timestamp"],infer_datetime_format=True)
act_daily["date"] = act_daily["timestamp"].dt.date
act_daily.head()

,timestamp,calories,bmr,steps,distance,sedentary_minutes,lightly_active_minutes,fairly_active_minutes,very_active_minutes,calories_from_activities,bmi,fat,weight,food_calories_logged,water_logged,beiwe,date
0,2020-05-13,2781.0,1876.0,9207,4.396294,1241,70,118,11,1097.0,23.754,0.0,180.0,0.0,0.0,hfttkth7,2020-05-13
1,2020-05-14,3727.0,1876.0,15207,7.261114,614,263,134,23,2234.0,23.754,0.0,180.0,0.0,0.0,hfttkth7,2020-05-14
2,2020-05-15,3909.0,1876.0,14556,8.028501,577,205,57,108,2381.0,23.754,0.0,180.0,0.0,0.0,hfttkth7,2020-05-15
3,2020-05-16,3927.0,1876.0,18453,8.748670,760,176,24,151,2364.0,23.754,0.0,180.0,0.0,0.0,hfttkth7,2020-05-16
4,2020-05-17,4180.0,1876.0,15425,7.973149,605,207,50,131,2652.0,23.754,0.0,180.0,0.0,0.0,hfttkth7,2020-05-17


In [9]:
act_hourly = pd.read_csv("../data/processed/fitbit-intraday-ux_s20.csv",parse_dates=["timestamp"],infer_datetime_format=True)
act_hourly.head()

,timestamp,calories,steps,distance,heartrate,beiwe
0,2020-05-13 17:39:00,1.69377,NaN,NaN,77.333333,hfttkth7
1,2020-05-13 17:40:00,1.69377,NaN,NaN,77.600000,hfttkth7
2,2020-05-13 17:41:00,4.42986,33.0,0.015721,79.000000,hfttkth7
3,2020-05-13 17:42:00,1.69377,NaN,NaN,83.416667,hfttkth7
4,2020-05-13 17:43:00,1.43319,NaN,NaN,63.666667,hfttkth7


### Fitbit Sleep

In [10]:
sleep_fb = pd.read_csv("../data/processed/fitbit-sleep_summary-ux_s20.csv",
                       parse_dates=["start_date","end_date","end_time","start_time"],infer_datetime_format=True)
sleep_fb["date"] = sleep_fb["end_date"].dt.date
sleep_fb.head()

,start_date,end_date,deep_count,deep_minutes,light_count,light_minutes,rem_count,rem_minutes,wake_count,wake_minutes,beiwe,efficiency,end_time,start_time,redcap,beacon,nrem_count,nrem_minutes,rem2nrem,tst_fb,rem_percent,nrem_percent,light_percent,deep_percent,waso,sol_fb,wol_fb,date
0,2020-05-14,2020-05-14,5.0,84.0,20.0,213.0,10.0,82.0,21.0,27.0,hfttkth7,93.349754,2020-05-14 07:13:00,2020-05-14 00:27:00,NaN,NaN,25.0,297.0,0.276094,6.316667,0.216359,0.783641,0.562005,0.221636,18.5,8.5,0.0,2020-05-14
1,2020-05-14,2020-05-15,4.0,95.0,31.0,250.0,6.0,47.0,33.0,101.0,hfttkth7,79.513185,2020-05-15 08:06:30,2020-05-14 23:53:30,NaN,NaN,35.0,345.0,0.136232,6.533333,0.119898,0.880102,0.637755,0.242347,68.5,0.0,32.5,2020-05-15
2,2020-05-15,2020-05-16,2.0,47.0,17.0,190.0,8.0,50.0,20.0,42.0,hfttkth7,87.234043,2020-05-16 04:57:00,2020-05-15 23:28:00,NaN,NaN,19.0,237.0,0.210970,4.783333,0.174216,0.825784,0.662021,0.163763,19.5,11.5,11.0,2020-05-16
3,2020-05-17,2020-05-17,5.0,78.0,21.0,242.0,11.0,83.0,25.0,44.0,hfttkth7,90.156600,2020-05-17 09:28:30,2020-05-17 02:01:30,NaN,NaN,26.0,320.0,0.259375,6.716667,0.205955,0.794045,0.600496,0.193548,23.0,12.5,8.5,2020-05-17
4,2020-05-18,2020-05-18,5.0,96.0,20.0,167.0,14.0,88.0,28.0,65.0,hfttkth7,84.375000,2020-05-18 07:20:00,2020-05-18 00:24:00,NaN,NaN,25.0,263.0,0.334601,5.850000,0.250712,0.749288,0.475783,0.273504,53.5,11.5,0.0,2020-05-18


---

<a id='preprocessing'></a>

# Pre-Processing

## Combining EMAs
Combining EMAs completed on the same day.

In [11]:
ema = ema_e.merge(ema_m,on=["beiwe","redcap","beacon","date"],suffixes=["_e","_m"])
ema.head()

,timestamp_e,beiwe,content_e,stress_e,lonely_e,sad_e,energy_e,redcap,beacon,date,timestamp_m,content_m,stress_m,lonely_m,sad_m,energy_m,tst,sol,naw,restful
0,2020-05-13 19:00:23,awa8uces,1.0,1.0,1.0,3.0,2.0,28,26.0,2020-05-13,2020-05-13 09:15:49,0.0,2.0,1.0,1.0,1.0,2.0,10.0,3.0,1.0
1,2020-05-13 19:21:32,lkkjddam,0.0,2.0,3.0,1.0,0.0,12,21.0,2020-05-13,2020-05-13 12:30:38,1,1.0,3.0,3.0,2.0,7.0,45.0,2.0,1.0
2,2020-05-13 19:30:38,rvhdl2la,2.0,1.0,0.0,0.0,1.0,29,NaN,2020-05-13,2020-05-13 09:43:27,1,1.0,2.0,3.0,0.0,5.3,5.0,2.0,2.0
3,2020-05-13 20:07:04,xxvnhauv,1.0,3.0,1.0,2.0,0.0,21,22.0,2020-05-13,2020-05-13 09:42:19,1,1.0,1.0,3.0,0.0,6.0,30.0,3.0,1.0
4,2020-05-13 20:25:07,xdbdrk6e,2.0,1.0,2.0,1.0,3.0,23,24.0,2020-05-13,2020-05-13 18:16:29,2,1.0,2.0,1.0,2.0,8.0,20.0,3.0,2.0


## Integrating Activity Data

In [12]:
def add_fb_activity(df,fb_df,fb_label="steps"):
    """
    Adds the given Fitbit metric to the provided df
    
    """
    fb_df["date"] = fb_df["timestamp"].dt.date
    merged = fb_df.merge(df,on=["beiwe","date"],how="inner",suffixes=["_fb","_ema"])
    data = {"beiwe":[],"date":[],fb_label:[]}
    for pt in merged["beiwe"].unique():
        merged_pt = merged[merged["beiwe"] == pt]
        for d in merged_pt["date"].unique():
            merged_pt_d = merged_pt[merged_pt["date"] == d]
            try:
                merged_pt_d.set_index("timestamp",inplace=True)
                merged_pt_d = merged_pt_d[:merged_pt_d["timestamp_e"][0]]
            except KeyError:
                merged_pt_d.set_index("timestamp_fb",inplace=True)
                merged_pt_d = merged_pt_d[:merged_pt_d["timestamp_ema"][0]]
            for key, val in zip(data.keys(),[pt,d,merged_pt_d[fb_label].sum()]):
                data[key].append(val)
                
    return df.merge(right=pd.DataFrame(data=data),on=["beiwe","date"])

In [13]:
def add_fb_percentage(df,df_fb):
    """
    Adds the percentage of time the participant was wearing their Fitbit based on the activity minutes per day
    
    """
    df_fb["active_minutes"] = df_fb[[col for col in df_fb.columns if col.endswith("minutes")]].sum(axis=1)
    df_fb["active_percent"] = df_fb["active_minutes"] / 1440
    
    return df.merge(right=df_fb[["beiwe","date","active_percent"]],on=["beiwe","date"])

In [14]:
temp = add_fb_activity(ema,act_hourly)
ema_activity = add_fb_percentage(temp,act_daily)

## Integrating Objective Sleep Data

In [15]:
def add_fb_sleep(df,df_fb):
    """
    Adds the objective sleep data from Fitbit to the give dataframe
    
    """
    return df.merge(right=df_fb,on=["beiwe","redcap","beacon","date"])

In [16]:
data = add_fb_sleep(ema_activity,sleep_fb)

<div class="alert alert-block alert-success">
    
We have 974 observations when combining all datasets together: previous mood, activity, and sleep before filtering of any kind.  
    
</div>

---

<a id='summary'></a>

# Summary
The following cells summarize the available data for analysis and save them to the `processed` directory for future analysis.

## Daily EMAs
We have actually already saved these data to a file. We can compare just to make sure.

In [17]:
emas = pd.read_csv("../data/processed/beiwe-daily_ema-ux_s20.csv")
print(len(ema) == len(emas))

True


<div class="alert alert-block alert-success">
    
Just considering the EMAs, we have a total of **2149** observations constituting both mood and morning sleep reports.
    
</div>

In [18]:
ema.to_csv("../data/interim/mood_prediction/beiwe-beiwe-ema_morning-ema_evening.csv")

## Activity and Evening EMA

In [19]:
act_ema_e = add_fb_activity(ema_e,act_hourly)

<div class="alert alert-block alert-success">
    
Now looking at the activity data and evening EMAs, we have **1999** observations.
    
</div>

In [20]:
act_ema_e.to_csv("../data/interim/mood_prediction/fitbit-beiwe-activity-ema_evening.csv")

## Sleep and Evening EMA

In [21]:
sleep_ema_e = add_fb_sleep(ema_e,sleep_fb)

<div class="alert alert-block alert-success">
    
We have **1108** observations.
    
</div>

In [22]:
sleep_ema_e.to_csv("../data/interim/mood_prediction/fitbit-beiwe-sleep-ema_evening.csv")

## Activity and EMAs

In [45]:
print("Number of Observations:",len(ema_activity))
print("Number of Pariticpants:",len(ema_activity["beiwe"].unique()))

Number of Observations: 1402
Number of Pariticpants: 51


<div class="alert alert-block alert-success">
    
We have **1402** observations.
    
</div>

In [24]:
ema_activity.to_csv("../data/interim/mood_prediction/fitbit-beiwe-beiwe-activity-ema_morning-ema_evening.csv")

## Sleep and EMAs

In [25]:
ema_sleep = add_fb_sleep(ema,sleep_fb)

<div class="alert alert-block alert-success">
    
We have **976** observations.
    
</div>

In [26]:
ema_sleep.to_csv("../data/interim/mood_prediction/fitbit-beiwe-beiwe-sleep-ema_morning-ema_evening.csv")

## Activity, Sleep, and Evening EMA

In [27]:
act_sleep_ema_e = add_fb_sleep(act_ema_e,sleep_fb)

<div class="alert alert-block alert-success">
    
We have **1106** observations - only two fewer than when we consider just sleep and the evening EMA.
    
</div>

In [28]:
act_sleep_ema_e.to_csv("../data/interim/mood_prediction/fitbit-fitbit-beiwe-activity-sleep-ema_evening.csv")

## Combined
All the datasets combined

In [29]:
print("Number of Observations:",len(data))

Number of Observations: 974


<div class="alert alert-block alert-success">
    
We have **974** observations - again just two fewer than when we include activity with the EMAs and sleep.
    
</div>

In [30]:
data.to_csv("../data/interim/mood_prediction/fitbit-fitbit-beiwe-beiwe-activity-sleep-ema_morning-ema_evening.csv")

---

[Back to Top](#top)